In [ ]:
# surpress warnings
import warnings
warnings.filterwarnings("ignore")

# README File

In [ ]:
# always generating a readme file at start (overwriting if it exists)
file = open("README.txt", 'w')
file.write("Please install the necessary modules/packages below in your command prompt by typing in the commands below.\n")
file.write("If you do not have pip installed please install it here: https://pip.pypa.io/en/stable/installation/" + "\n")
file.write("The only file that is needed that is NOT generated by this program is the client_secret.txt file" + "\n")
file.write("This file contains important authentication data to verify our identity." + "\n\n")

module_imports = (
"requests - pip install requests\n" +  
"time - pip install time\n" +
"datetime - pip install datetime\n" +
"bs4 - pip install bs4\n" +
"re - pip install re\n" + 
"pandas - pip install pandas\n" +
"pydrive - pip install pydrive\n" + 
"numpy - pip install numpy\n" +
"selenium - pip install selenium")
file.write(module_imports + "\n\n")
file.write("Enjoy!")
file.close()

# Imports

In [ ]:
# if any of the modules are not installed
# please install this in the terminal

import requests # to access, login, the acuity website
import time # for time delay
from datetime import datetime # to access the current time (military time)
from datetime import date # to found the current time and compare it to others
from bs4 import BeautifulSoup
import re # regular expressions
import random # random delays for events
import pandas as pd# for reading the data
from pydrive.auth import GoogleAuth # google drive access
from pydrive.drive import GoogleDrive # google drive access
from io import BytesIO # to access to downloaded
import json # to load json file
import os # to delete the downloaded file
import numpy as np # vectorization
from selenium import webdriver # pip install selenium
from selenium.webdriver.common.by import By # to find elements
from selenium.webdriver.support.ui import WebDriverWait # to wait for elements
from selenium.webdriver.support import expected_conditions as EC # exception handling
from selenium.webdriver.common.keys import Keys # to press buttons like enter, tab, backspace, and etc...
# if any of the modules are not installed
# please install this in the terminal

# Functions

In [ ]:
def log(string):
    '''
    Creates a function to log all the needed information
    Prints and logs information to the log.txt file with todays date
    '''
    
    the_time = "[" + str(datetime.now()).split()[1].split(".")[0][:-3] + "]"
    # opening, writing, and closing the file
    file = open(str(date.today().strftime("%B %d, %Y").replace(",","").replace(" ","-")) + '-log.txt', 'a+')
    file.write(string +  " " + the_time + " " + "\n")
    file.close()
    
    # we also print a string of the information that was saved to the console
    print(string)
    
def error(error_message):
    '''Ends the program after 15 seconds if there is an error'''
    
    global driver
    
    # ALWAYS FREE UP MEMORY AFTER ANY ERROR OR APPLICATION CLOSE
    try:
        driver.quit()
    except:
        pass
    
    log(error_message)
    log("Ending program in 15 seconds.")
    time.sleep(15.314159)
    exit()
    
def google_authenticate():

    """
    Authenticates access to google drive from our application.
    LOGIN with mathnasiumathome@mathnasium.com if prompted
    IF we are already authenticated, then we do not have to authenticate again
    
    Returns - drive, associated with the login credentials to access to google drive
    """
    
    # write a fake json file with fake keys and automagically reauthenticate it
    json_information = {"Not revealing the authentication key on github"}
    file = open("client_secrets.txt","w")
    file.write(fake_json_information)
    file.close()
    gauth = GoogleAuth()
    # Try to load saved client credentials
    gauth.LoadCredentialsFile("client_secrets.txt")
    if gauth.credentials == None:
        # Authenticate if they're not there
        gauth.LocalWebserverAuth()
    elif gauth.access_token_expired:
        # Refresh them if expired
        gauth.Refresh()
    else:
        # Initialize the saved creds
        gauth.Authorize()
    # Save the current credentials to a file
    gauth.SaveCredentialsFile("client_secrets.txt")

    drive = GoogleDrive(gauth)

    return drive

def read_file_from_drive(file_name, file_extension, folder_id):
    """
    file_name - a string containing the file name without the extension
    file_extension - a string containing file extension 
    folder_id - a string containing the folder id
    
    Reads the specified file in the 
    correct drive folder
    
    Returns - data, of the file read in the drive folder
    """

    # authentication for google drive
    try:
        drive = google_authenticate()
    except Exception as e:
        log("Could not login to google.")
        error(str(e))

    # file type downloads
    mimetypes = {'application/vnd.google-apps.document': 'application/pdf',
                 'application/vnd.google-apps.spreadsheet': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'}
    # by default
    download_mimetype = None

    # getting files in folder
    file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()

    # iterating through the files until we reach the one we want to download 
    for file in file_list:
        # downloading the file
        if file['mimeType'] in mimetypes and file['title'] == file_name:
            download_mimetype = mimetypes[file['mimeType']]
            # using the mimeType to download the correct file
            file.GetContentFile(file['title'] + file_extension, mimetype = download_mimetype)
            
            if file_extension == ".xlsx":
                file_data = pd.read_excel(file_name + file_extension)
                os.remove(file_name + file_extension)
                return (file_data, True)
            
    return (False, False)
            
def read_config():
    """
    Attempts to read out config file
    to grab the username and password to sign 
    into the website. If this fails, we will present
    an error code and the program will stop.
    
    If successful we return the supplemented username
    as password of the user.
    """
    try:
        file = open('config.txt','r')
    except Exception as FileNotFoundError:
        log("The config file has not been created.")
        log("Creating the config file...")
        
        # creating our config file
        file = open('config.txt','w')
        
        log("config.txt file successfully created!")
        log("Please enter credentials to access the website.")
        log("This information will be saved in the config file.")
        username = input("Username: ")
        password = input("Password: ")
        
        # writing credentials
        with open('config.txt', 'w') as f:
            f.write("Username: " + username + "\n" + "Password: " + password)
            
        log("config.text successfully saved with new credentials.")
        
        # for accessing our data
        file = open('config.txt','r')
        
    try:
        # getting the data of the config file
        data = file.readlines()

        login_information = []
        for credentials in data:
            login_information.append(credentials.split(":")[1].strip())

        # we return a tuple, so this information remains unmodified
        log("Detected username: " + str(login_information[0]))
        log("Detected password: " + len(str(login_information[1]))*"*")
        
    except Exception as e:
        log("Error reading the config file. " + str(e))
        log("Please delete the config file and try again.")
        file.close()
        error("Could not read the credentials of the config file.")
        
    log("Successfully read user credentials from config.txt file.")
    file.close()
    return tuple(login_information)

def click_XPATH(xpath):
    
    global driver
    global options
    
    """
    xpath - destination of element via XPATH 
    Waits and clicks for the button to appear
    Grace period is about 10 seconds before the program
    quits itself
    """
    
    if options.headless == True:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath)))
        button = driver.find_element(By.XPATH, xpath)
        driver.execute_script("arguments[0].click();", button)
    else:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath))).click()

def login_to_text_free_us():

    global driver
    global options
    # HEADLESS BROWSER - more light weight and makes this seem more like magic 
    options = webdriver.ChromeOptions()
    options.headless = True
    options.add_argument("--log-level=3")
    
    log("Openning webdriver...")
    driver_versions = ["100","101","102","103","104","105","106","107","108"]
    for driver in driver_versions:
        try:
            driver = webdriver.Chrome('./chromedriver' + driver, options = options)
            break
        except:
            driver = None
        
    # if the driver is still none then we know that we could not find a supported version of chrome
    if driver == None:
        error("Could not find a proper version chrome driver to use.\nPlease visit https://chromedriver.chromium.org/downloads and download the correct version.\n" + 
              "Once downloaded name the rename the file to chromedriver{version}.exe and move it into the folder with the program.")
        
    driver.get('https://messages.textfree.us/login')
    log("Successfully opened webbrowser!")
    log("Loaded login page.")

    try:
        log("Attempting to login to textfree.us")

        # filling in the username field
        username_xpath = '//*[@id="username"]'
        click_XPATH(username_xpath)
        driver.find_element(By.XPATH, username_xpath).send_keys(username)
        log("Entered username.")

        # filling in the password field
        password_xpath = '//*[@id="password"]'
        click_XPATH(password_xpath)
        driver.find_element(By.XPATH, password_xpath).send_keys(password)
        log("Entered password.")

        # submitting login information
        click_XPATH('//*[@id="loginForm"]/div[4]/button')
        log("Submitting information.")
        log("Successfully submitted!")
        log("Verifying the webpage's identity...")
    except Exception as e:
        print(str(e))
        error("Could not login to textfree.us")

    # small delay
    time.sleep(8)
    if 'conversation' in driver.current_url:
        log("Verified!")
        log("Successfully logged in!")
    else:
        error("Could not login to textfree.us")

    try:
        click_XPATH('//*[@id="SyncContactsXDismissPopup"]')
        log("Successfully closed the google popup window!")    
    except:
        log("There was no popup window to close.")    
        
def enter_phone_numbers(phone_numbers):
    """
    Enters phone number after login.
    
    Paramters:
    phone_numbers - a list of phone numbers
    based on the parent's name in the google sheet
    """
    
    global driver
    
    try:
        # start a new conversation
        click_XPATH('//*[@id="startNewConversationButton"]')
        click_XPATH('//*[@id="contactInput"]')

        # enter the phone numbers associated with the name of the parents
        for number in phone_numbers:
            driver.find_element(By.XPATH, '//*[@id="contactInput"]').send_keys(number)
            driver.find_element(By.XPATH, '//*[@id="contactInput"]').send_keys(Keys.ENTER)
            
            
    except Exception as e:
        error("Failed to enter phone numbers!")
        
def read_reminder_message():
    """
    This function reads the reminder message
    contained in the message.txt file
    
    Returns: A string of the reminder message provided
    """
    
    try:
        # opening the file
        file = open('message.txt', 'r')
        message_str = "" # our initial message
        start = False # ignore everything until we need to read the message

        for line in file.readlines():
            if "Reminder Message:" in line:
                start = True
            if start:
                message_str += line.replace("\n","").replace("\t","")

        # closing file
        file.close()

        return message_str.replace("Reminder Message:","").strip()
    except:
        error("Could not read the reminder message!")
        
def get_similar_names(first_name = "!",last_name = "!"):
    """
    Gets information based on the provided names
    """
    
    global df
    
    first_name = first_name.lower()
    last_name = last_name.lower()
    
    query = []
    # getting a row in the google sheet
    for row in df.iterrows():
        index = row[0]
        ser = row[1]
        
        if (first_name in ser["Student First Name"].lower() or last_name in ser['Student Last Name'].lower()  or
        last_name in ser["Student First Name"].lower()  or first_name in ser['Student Last Name'].lower() ):
            query.append([(ser['Student First Name'],ser['Student Last Name'],ser['Parent First Name']),ser])

    return query

def send_message(message, row):
    """
    Sends the message to the user
    
    Paramters:
    message - a string that is provided 
    by the user in the message.text file
    
    row - a row from the google sheet
    that will allow us to modify the message accordingly
    and send the correct text messages to the parents
    
    """
    
    # getting information from a specified row in the google sheets
    
    
    # select the box to start typing
    try:
        # entering the parents phone numbers
        enter_phone_numbers(row["Parent(s) Cell Phone"])
        
        textfield_XPATH = '/html/body/div[1]/conversation-container/div/div/main/div[2]/ng-transclude/div/div[2]/conversation-input/form/div/div[2]/div[1]/div'
        click_XPATH(textfield_XPATH)
        driver.find_element(By.XPATH, textfield_XPATH).send_keys(message)
        time.sleep(1)
        driver.find_element(By.XPATH, textfield_XPATH).send_keys(Keys.ENTER)
    except:
        error("Could not enter and send message!")
    
    log(f"Successfully sent the reminder to {row['Student First Name']} {row['Student Last Name']}'s parent(s).")
    
def get_google_data():
    
    log("Program is starting (this will take a minute)...")
    
    """
    Downloads data from google drive.
    Student/Parent/Phone/Email info.
    """
    global username
    global password
    global reminder_message
    global driver
    global df
    
    try:
        df = read_file_from_drive("Remind List",".xlsx","1bS9zj9c3Tst270cbPv2EYr1sqzrGs0XS")
    except Exception as e:
        error("Error Accessing Google: " + str(e))

    if df[1] == False:
        error("Failed to retrieve the specified file from google drive.")

    df = df[0]

    # people might have multiple phone numbers or emails
    new_phone_numbers = []
    new_emails = []
    for row in df.iterrows():

        index = row[0]
        ser = row[1]

        parent_cell_phone_numbers = str(ser["Parent(s) Cell Phone"])
        if "," in parent_cell_phone_numbers:
            new_phone_numbers.append([x for x in parent_cell_phone_numbers.split(",") if len(x) == 10 or len(x) == 1])
        elif len(parent_cell_phone_numbers) == 10 or len(parent_cell_phone_numbers) == 11:
            new_phone_numbers.append([parent_cell_phone_numbers])
        else:
            new_phone_numbers.append([np.nan])

        parents_email = str(ser["Parent(s) Email"])
        if "," in parents_email:
            new_emails.append([x for x in parents_email.split(",") if "@" in x])
        elif "@" in parents_email:
            new_emails.append([parents_email])
        else:
            new_emails.append([np.nan])

    df["Parent(s) Cell Phone"] = new_phone_numbers
    df["Parent(s) Email"] = new_emails

#     addon = pd.DataFrame({"Student First Name": ["Test"],
#                   "Student Last Name": ["test!"],
#                   "Parent First Name":["Vader"],
#                   "Parent(s) Cell Phone":[["45454545"]],
#                   "Parent(s) Email":[['aaa@gmail.com']]})
#     df = df.append(addon, ignore_index = True)
    
def read_files_and_launch():
    """
    Reads the config and message
    files to ensure proper security and connection
    while running the program.
    """

    global username
    global password
    global reminder_message
    global driver
    global df
    
    username, password = read_config() # access textfree.us
    reminder_message = read_reminder_message()
    if len(reminder_message) < 5:
        error("Please provide a reminder message in message.txt, then restart the program.")
    else:
        log("Successfully read the reminder message.")
        log("Reminder message: " + str(reminder_message))

    # logging in
    login_to_text_free_us()
    log("Program is ready.\n")
    
def remind(user_input):
    """
    The remind function. 
    Uses gathered data from google and textfree.us
    to send a reminder message to the parent.
    """
    
    global username
    global password
    global reminder_message
    global driver
    global df
    
    driver.refresh()
    
    success = False
    commands = user_input.strip().split(" ")
    while success == False:
        nums_str = ['1','2','3','4','5','6','7','8','9']
        if len(commands) == 3:
            if (commands[1][0] in nums_str) and (commands[-1][0] not in nums_str):
                first_name = commands[0]
                last_name = commands[1]
                session_time = commands[1]
                location = commands[-1]
                break
                success = True
            else:
                log("Invalid format. Please try swapping the time with the location.")
        elif len(commands) == 4:
            if (commands[2][0] in nums_str) and (commands[-1][0] not in nums_str):
                first_name = commands[0]
                last_name = commands[1]
                session_time = commands[2]
                location = commands[-1]
                break
                success = True
            else:
                log("Invalid format. Please try swapping the time with the location.")
        else:
            log("Invalid input! Please type the information in the correct format: NAME TIME LOCATION")
            
        command = input("Please enter the STUDENT'S FIRST NAME or the STUDENT'S LAST NAME or both AND a SESSION TIME AND LOCATION with spaces in between: ").strip()
        commands = command.split(" ")



    possible_students = get_similar_names(first_name,last_name)
    log("\nNumber of students found: " + str(len(possible_students)))
    if len(possible_students) > 0:
        log("STUDENT SELECTION: Please choose a student based on the numbers provided.")

        for student in enumerate(possible_students):
            print("(" + str(student[0]+1) + ")",student[1][0][0],student[1][0][1],session_time,location)

        success = False
        while success == False and len(possible_students) != 1:

            try:
                student_num = int(input("Enter a student number: "))

                if student_num <= 0 or student_num > len(possible_students):
                    log("Invalid student number. Please try again.")
                else:
                    success = True
            except:
                log("Invalid student number. Please try again.")

        # for auto selection of student if there was exactly one found
        if len(possible_students) == 1:
            student_num = 0
        else:
            student_num -= 1

        first_name = possible_students[student_num][0][0]
        last_name = possible_students[student_num][0][1]
        row = possible_students[student_num][1]

        if np.nan not in row:
            # getting confirmation
            success = False
            while success == False:
                try:
                    confirmation = input("CONFIRMATION: Would you like to send a reminder to " +
                        f"{first_name} {last_name}'s parent(s)? \ny/n ").lower()

                    if confirmation in ['y','n','no','yes']:
                        success = True
                        break
                    log("Try again. Please enter y for yes or n for no.")
                except:
                    log("Please enter y/n")

            # LET'S SEND IT
            if confirmation in ['y','yes']:
                send_message(reminder_message.replace("{parent_name}",row['Parent First Name']).replace("{student_name}",first_name).replace(
                "{session_time}",session_time).replace("{location}",location).replace("@H","@Home").replace("IC","In-Center"),row)
            else:
                log("Message not sent!")
        else:
            log("Detected missing information. Cannot send the reminder.")
        
def main():
    """
    The main program.
    """

    # INITIALIZATION
    get_google_data() # getting data
    read_files_and_launch() # launching the browser

    # PROGRAM START
    while True:
        try:
            user_input = input("Ready: (@quit to quit):\n")

            if len(user_input.split(" ")) > 2:
                remind(user_input)
            elif user_input[:5].lower() == "@quit":
                driver.quit()
                log("Goodbye!")
                time.sleep(10)
                exit()
                break
            elif user_input == 'oh no!':
                log("What's wrong?! Are you ok? How was your day today? I am sure it's fine! You're the best!")
            else:
                log("Invalid command!")

        except Exception as e:
            log("An error has occured within the main of the program.")
            log("Try restarting the application or contact Mr. Harrison.")
            driver.quit()        
            error(str(e))

In [ ]:
if __name__ == "__main__":
    main()